In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_ta as ta
import re
import requests

from datetime import datetime

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [2]:
from invaas.schwab.schwab_task import SchwabTask

task = SchwabTask(env="local")
await task.setup_api_and_get_products()

2024-01-28 22:32:30 WARNING No DBUtils defined in the runtime
2024-01-28 22:32:30 INFO Initializing task for local environment
2024-01-28 22:32:30 INFO Historical period used for analysis: 14 days
2024-01-28 22:32:30 INFO Min buy amount: $5
2024-01-28 22:32:30 INFO Max buy amount: $100
2024-01-28 22:32:30 INFO Logging in to Schwab


In [3]:
def get_options_chains_data(ticker: str):
    options_info = requests.get(f"https://client.schwab.com/symlup/OptionsCwp/Underlying/{ticker}.txt").text
    options_info = re.sub(r"[\n\t\r]*", "", options_info)
    options_info = json.loads(options_info.replace('SuggestionBox.JsonpCallback("option",', "")[:-1])
    expirations = [x["Date"] for x in options_info["Expirations"]]
    options_chain_url = f"https://ausgateway.schwab.com/api/is.CSOptionChainsWeb/v1/OptionChainsPort/OptionChains/chains?Symbol={ticker}&IncludeGreeks=false&{','.join(expirations)}"
    task.schwab_api.update_token("update")
    return json.loads(requests.get(options_chain_url, headers=task.schwab_api.headers).text)


options_chains_data = get_options_chains_data("QQQ")
options_chains_data

{'UnderlyingData': {'Ask': '423.3',
  'AskSize': '1',
  'Bid': '423.22',
  'BidSize': '3',
  'Close': '426.35',
  'CumulativeVolume': '37137028',
  'Description': 'INVSC QQQ TRUST SRS 1',
  'ErrorCode': 0,
  'InstrumentType': 'ETF',
  'Last': '423.81',
  'NetChange': '-2.54',
  'PercentChange': '-0.6',
  'Open': '424.44',
  'DailyHigh': '426.21',
  'DailyLow': '423.15',
  'High52W': '429.85',
  'Low52W': '285.19',
  'Symbol': 'QQQ',
  'DividendYield': 0.5,
  'ExchangeName': 'NASDAQ',
  'IsDelayed': False},
 'Expirations': [{'ExpirationGroup': {'RootSymbol': 'QQQ',
    'MonthAndDay': 'Jan 29',
    'Year': '2024',
    'Day': 'Mon',
    'SettlementType': '',
    'DaysUntil': 1},
   'AdjustedLabel': {},
   'AdjustedGroup': '',
   'Chains': [{'SymbolGroup': 'QQQ20240129353',
     'Legs': [{'Sym': 'QQQ   240129C00353000',
       'OptionType': 'C',
       'Strk': '353',
       'Chg': '0.08',
       'ChgPct': '0.11',
       'Bid': '70.12',
       'Ask': '70.54',
       'Lst': '71.67',
       '

In [9]:
data = []
underlying_last = float(options_chains_data["UnderlyingData"]["Last"])

for expiration in options_chains_data["Expirations"]:
    days_until_expiration = expiration["ExpirationGroup"]["DaysUntil"]
    expiration_month_day = expiration["ExpirationGroup"]["MonthAndDay"].split(" ")
    expiration_month = expiration_month_day[0]
    expiration_day = expiration_month_day[1]
    expiration_year = expiration["ExpirationGroup"]["Year"]
    expiration_date = datetime.strptime(f"{expiration_month} {expiration_day} {expiration_year}", "%b %d %Y")

    for chain in expiration["Chains"]:
        data_obj = {}
        data_obj["OPTION_ID"] = chain["SymbolGroup"]
        data_obj["UNDERLYING_LAST"] = underlying_last
        data_obj["EXPIRE_DATE"] = expiration_date
        data_obj["DTE"] = days_until_expiration

        for leg in chain["Legs"]:
            data_obj["STRIKE"] = float(leg["Strk"])
            data_obj["STRIKE_DISTANCE"] = abs(data_obj["STRIKE"] - underlying_last)
            data_obj["STRIKE_DISTANCE_PCT"] = data_obj["STRIKE_DISTANCE"] / underlying_last
            leg_prefix = leg["OptionType"] + "_"
            data_obj[leg_prefix + "ID"] = leg["Sym"]
            data_obj[leg_prefix + "BID"] = float(leg["Bid"])
            data_obj[leg_prefix + "ASK"] = float(leg["Ask"])
            data_obj[leg_prefix + "VOLUME"] = int(leg["Vol"])

        data.append(data_obj)

df_options_chains = pd.DataFrame(data=data)
df_options_chains

,OPTION_ID,UNDERLYING_LAST,EXPIRE_DATE,DTE,STRIKE,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,C_ID,C_BID,C_ASK,C_VOLUME,P_ID,P_BID,P_ASK,P_VOLUME
0,QQQ20240129353,423.81,2024-01-29,1,353.0,70.81,0.167080,QQQ 240129C00353000,70.12,70.54,1,QQQ 240129P00353000,0.00,0.01,0
1,QQQ20240129354,423.81,2024-01-29,1,354.0,69.81,0.164720,QQQ 240129C00354000,69.12,69.54,0,QQQ 240129P00354000,0.00,0.01,0
2,QQQ20240129355,423.81,2024-01-29,1,355.0,68.81,0.162360,QQQ 240129C00355000,68.12,68.55,11,QQQ 240129P00355000,0.00,0.01,0
3,QQQ20240129356,423.81,2024-01-29,1,356.0,67.81,0.160001,QQQ 240129C00356000,67.12,67.55,1,QQQ 240129P00356000,0.00,0.01,0
4,QQQ20240129357,423.81,2024-01-29,1,357.0,66.81,0.157641,QQQ 240129C00357000,66.12,66.55,0,QQQ 240129P00357000,0.00,0.01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4153,QQQ20261218620,423.81,2026-12-18,1055,620.0,196.19,0.462920,QQQ 261218C00620000,10.00,15.00,3,QQQ 261218P00620000,194.11,199.00,0
4154,QQQ20261218625,423.81,2026-12-18,1055,625.0,201.19,0.474717,QQQ 261218C00625000,9.50,14.50,4,QQQ 261218P00625000,199.11,204.00,0
4155,QQQ20261218630,423.81,2026-12-18,1055,630.0,206.19,0.486515,QQQ 261218C00630000,8.92,13.00,10,QQQ 261218P00630000,204.10,209.00,0
4156,QQQ20261218635,423.81,2026-12-18,1055,635.0,211.19,0.498313,QQQ 261218C00635000,8.24,11.96,71,QQQ 261218P00635000,209.10,214.00,0


In [10]:
from invaas.schwab.cnn_fear_greed_index import get_historical_cnn_fear_greed_index


def get_fear_greed_index_data():
    historical_period = 14
    df = pd.DataFrame(data=get_historical_cnn_fear_greed_index()["data"])
    df.set_index(
        pd.DatetimeIndex([pd.Timestamp(x, unit="ms", tz="UTC") for x in df.x]),
        inplace=True,
    )
    df.rename(columns={"y": "fear_greed_index"}, inplace=True)
    df.sort_index(inplace=True)
    df["fear_greed_index"] = df.fear_greed_index.fillna(method="ffill").astype(float)
    df["previous_max_fear_greed_index"] = (
        df["fear_greed_index"].rolling(window=historical_period, min_periods=historical_period).max()
    )
    df["previous_min_fear_greed_index"] = (
        df["fear_greed_index"].rolling(window=historical_period, min_periods=historical_period).min()
    )
    return (
        int(df.fear_greed_index.values[-1]),
        int(df.previous_max_fear_greed_index.values[-1]),
        int(df.previous_min_fear_greed_index.values[-1]),
    )


current_fear_greed_index, previous_max_fear_greed_index, previous_min_fear_greed_index = get_fear_greed_index_data()

print(f"Current fear greed index: {current_fear_greed_index}")
print(f"Previous max fear greed index: {previous_max_fear_greed_index}")
print(f"Previous min fear greed index: {previous_min_fear_greed_index}")
print()

good_call_buy = current_fear_greed_index >= previous_max_fear_greed_index - 1
good_put_buy = current_fear_greed_index <= previous_min_fear_greed_index + 1

print(f"Buy call: {good_call_buy}")
print(f"Buy put: {good_put_buy}")

Current fear greed index: 76
Previous max fear greed index: 77
Previous min fear greed index: 61

Buy call: True
Buy put: False


In [11]:
min_dte = 14
min_volume = 100
max_strike_distance_pct = 0.05
max_buy_price = 100

df_options_chains.loc[
    (df_options_chains.DTE > min_dte)
    & (df_options_chains.STRIKE_DISTANCE_PCT < max_strike_distance_pct)
    & (
        (good_call_buy & (df_options_chains.C_VOLUME > min_volume) & (df_options_chains.C_ASK * 100 <= max_buy_price))
        | (good_put_buy & (df_options_chains.P_VOLUME > min_volume) & (df_options_chains.P_ASK * 100 <= max_buy_price))
    )
].sort_values(by="STRIKE_DISTANCE_PCT")

,OPTION_ID,UNDERLYING_LAST,EXPIRE_DATE,DTE,STRIKE,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,C_ID,C_BID,C_ASK,C_VOLUME,P_ID,P_BID,P_ASK,P_VOLUME
1484,QQQ20240216444.78,423.81,2024-02-16,19,444.78,20.97,0.049480,QQQ 240216C00444780,0.81,0.82,132,QQQ 240216P00444780,21.50,21.77,2
1485,QQQ20240216445,423.81,2024-02-16,19,445.00,21.19,0.049999,QQQ 240216C00445000,0.79,0.80,2090,QQQ 240216P00445000,21.71,21.98,1
